In [ ]:
!git clone https://github.com/pizadi/ISIC2018-training.git

In [1]:
from os import chdir
chdir('./ISIC2018-training')

In [2]:
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Validation_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Validation_GroundTruth.zip
# !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Test_Input.zip

--2023-02-28 10:14:31--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 54.231.159.25, 54.231.196.89, 52.217.230.81, ...
Connecting to isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)|54.231.159.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11165358566 (10G) [application/zip]
Saving to: ‘ISIC2018_Task1-2_Training_Input.zip’

ISIC2018_Task1-2_Tr 100%[===================>]  10.40G  33.8MB/s    in 6m 16s  

2023-02-28 10:20:47 (28.3 MB/s) - ‘ISIC2018_Task1-2_Training_Input.zip’ saved [11165358566/11165358566]

--2023-02-28 10:20:47--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 52.217.4.28, 54.231.165.65, 52.216.76.140, ...
Connecting to isic-challenge-data.s3.amazonaws.com

In [ ]:
!unzip ./ISIC2018_Task1-2_Training_Input.zip
!rm ./ISIC2018_Task1-2_Training_Input.zip
!unzip ./ISIC2018_Task1_Training_GroundTruth.zip
!rm ./ISIC2018_Task1_Training_GroundTruth.zip
!unzip ./ISIC2018_Task1-2_Validation_Input.zip
!rm ./ISIC2018_Task1-2_Validation_Input.zip
!unzip ./ISIC2018_Task1_Validation_GroundTruth.zip
!rm ./ISIC2018_Task1_Validation_GroundTruth.zip
!mkdir ./Preproc

In [5]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import torch
from torch import nn as nn
from torch import functional as F
import cv2 as cv
import numpy as np
import torchvision
from tqdm import tqdm
from matplotlib import pyplot as plt
import torchinfo

In [3]:
TRAIN_INPUT_DIR = './ISIC2018_Task1-2_Training_Input/'
TRAIN_GT_DIR = './ISIC2018_Task1_Training_GroundTruth/'

VAL_INPUT_DIR = './ISIC2018_Task1-2_Validation_Input/'
VAL_GT_DIR = './ISIC2018_Task1_Validation_GroundTruth/'

BATCH_SIZE = 5
LEARNING_RATE = 1e-5

EPOCHS = 100
IM_H, IM_W = 384, 512

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")

Device: cuda


In [4]:
training_input_files = os.listdir(TRAIN_INPUT_DIR)
for filename in training_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    training_input_files.remove(filename)
n_train = len(training_input_files)

val_input_files = os.listdir(VAL_INPUT_DIR)
for filename in val_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    val_input_files.remove(filename)
n_val = len(val_input_files)
n_train -= n_train % BATCH_SIZE
n_val -= n_val % BATCH_SIZE
print(n_train, n_val)

2590 100


In [5]:
def train(model, stage):
  num_batches = n_train // BATCH_SIZE
  model.train()
  t_loss, t_met, proc = 0, torch.tensor([0., 0., 0., 0.]), 0
  with tqdm(total=num_batches) as pbar:
    pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
    for batch in range(num_batches):
      X = torch.load(f"./Preproc/train_X_{batch}.torch")
      y = torch.load(f"./Preproc/train_y_{batch}.torch")
      loss, met = None, None
      if stage == 0:
          loss, met = model.fit0(X, y)
      elif stage == 1:
          loss, met = model.fit1(X, y)
      elif stage == 2:
          loss, met = model.fit2(X, y)
      t_loss += loss*len(X)
      t_met += met
      proc += len(X)
      pbar.update(1)
      pbar.set_description(f"Avg. Loss: {t_loss/proc: .4f}, Avg. Accuracy: {(t_met[0] + t_met[3])/proc: .4f}")
  return (t_loss/n_train, t_met/n_train)

def test(model, stage):
  num_batches = n_val // BATCH_SIZE
  model.eval()
  t_loss, t_met, proc = 0, torch.tensor([0., 0., 0., 0.]), 0
  with tqdm(total=num_batches) as pbar:
    pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
    with torch.no_grad():
      for batch in range(num_batches):
        X = torch.load(f"./Preproc/val_X_{batch}.torch")
        y = torch.load(f"./Preproc/val_y_{batch}.torch")
        loss, met = None, None
        if stage == 0:
            loss, met = model.test0(X, y)
        elif stage == 1:
            loss, met = model.test1(X, y)
        elif stage == 2:
            loss, met = model.test2(X, y)
        t_loss += loss*len(X)
        t_met += met
        proc += len(X)
        pbar.update(1)
        pbar.set_description(f"Avg. Loss: {t_loss/proc : .4f}, Avg. Accuracy: {(t_met[0] + t_met[3])/proc: .4f}")
  return (t_loss/n_val, t_met/n_val)

In [6]:
train_X = torch.zeros((BATCH_SIZE, 3, IM_H, IM_W)).to(DEVICE)
train_y = torch.zeros((BATCH_SIZE, 1, IM_H, IM_W)).to(DEVICE)
nb = n_train//BATCH_SIZE
with tqdm(total=n_train) as pbar:
    for f in range(nb):
        l, r = f*BATCH_SIZE, (f+1)*BATCH_SIZE
        filename = None
        for i in range(l, r):
            filename = training_input_files[i]
            in_image = torchvision.io.read_image(TRAIN_INPUT_DIR + filename).to(DEVICE)
            gt_image = torchvision.io.read_image(TRAIN_GT_DIR + filename[:-4] + "_segmentation.png").to(DEVICE)
            train_X[i-l, :, :, :] = torchvision.transforms.functional.resize(in_image, [IM_H, IM_W])
            train_y[i-l, :, :, :] = torchvision.transforms.functional.resize(gt_image, [IM_H, IM_W])
            in_image, gt_image = None, None
            torch.cuda.empty_cache()
            pbar.update(1)

        train_y = (train_y//128)
        torch.save(train_X, f"./Preproc/train_X_{f}.torch")
        torch.save(train_y, f"./Preproc/train_y_{f}.torch")
train_X, train_y = None, None
torch.cuda.empty_cache()
   
val_X = torch.zeros((BATCH_SIZE, 3, IM_H, IM_W)).to(DEVICE)
val_y = torch.zeros((BATCH_SIZE, 1, IM_H, IM_W)).to(DEVICE)
nb = n_val//BATCH_SIZE
with tqdm(total=n_val) as pbar:
    for f in range(nb):
        l, r = f*BATCH_SIZE, (f+1)*BATCH_SIZE
        filename = None
        for i in range(l, r):
            filename = val_input_files[i]
            in_image = torchvision.io.read_image(VAL_INPUT_DIR + filename).to(DEVICE)
            gt_image = torchvision.io.read_image(VAL_GT_DIR + filename[:-4] + "_segmentation.png").to(DEVICE)
            val_X[i-l, :, :, :] = torchvision.transforms.functional.resize(in_image, [IM_H, IM_W])
            val_y[i-l, :, :, :] = torchvision.transforms.functional.resize(gt_image, [IM_H, IM_W])
            in_image, gt_image = None, None
            torch.cuda.empty_cache()
            pbar.update(1)

        val_y = (val_y//128)
        torch.save(val_X, f"./Preproc/val_X_{f}.torch")
        torch.save(val_y, f"./Preproc/val_y_{f}.torch")
val_X, val_y = None, None
torch.cuda.empty_cache()


KeyboardInterrupt: ignored

In [7]:
def centroid(I):
    p = torch.ones(I.shape).to(I.device.type)
    ipos = torch.cumsum(p, axis=0) - 1
    jpos = torch.cumsum(p, axis=1) - 1
    return (torch.sum(ipos*I)/torch.sum(I)).item() , (torch.sum(jpos*I)/torch.sum(I)).item()

def _centroid(img, lcc=False):
  if lcc:
    img = img.astype(np.uint8)
    nb_components, output, stats, centroids = cv.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    if len(sizes) > 2:
      max_label = 1
      max_size = sizes[1]

      for i in range(2, nb_components):
          if sizes[i] > max_size:
              max_label = i
              max_size = sizes[i]

      img2 = np.zeros(output.shape)
      img2[output == max_label] = 255
      img = img2

  if len(img.shape) > 2:
    M = cv.moments(img[:,:,1])
  else:
    M = cv.moments(img)

  if M["m00"] == 0:
    return (img.shape[0] // 2, img.shape[1] // 2)
  
  cX = int(M["m10"] / M["m00"])
  cY = int(M["m01"] / M["m00"])
  return (cX, cY)

def to_polar(input_img, center):
  input_img = input_img.astype(np.float32)
  value = np.sqrt(((input_img.shape[0]/2.0)**2.0)+((input_img.shape[1]/2.0)**2.0))
  polar_image = cv.linearPolar(input_img, center, value, cv.WARP_FILL_OUTLIERS)
  return polar_image

def to_cart(input_img, center):
  input_img = input_img.astype(np.float32)
  value = np.sqrt(((input_img.shape[1]/2.0)**2.0)+((input_img.shape[0]/2.0)**2.0))
  polar_image = cv.linearPolar(input_img, center, value, cv.WARP_FILL_OUTLIERS + cv.WARP_INVERSE_MAP)
  polar_image = polar_image.astype(np.uint8)
  return polar_image

In [8]:
def torch2np(I):
    J = None
    if (I.dim() == 3):
        J = I.transpose(0, 1).transpose(1, 2).detach().to("cpu").numpy()
    elif (I.dim() == 2):
        J = I.detach().to("cpu").numpy()
    return J

In [9]:
def np2torch(I):
    J = None
    if (I.ndim == 3):
        J = torch.tensor(I).transpose(1, 2).transpose(0, 1)
    elif (I.ndim == 2):
        J = torch.tensor(I)[None,:,:]
    return J

In [10]:
class UNetRes50(nn.Module):
    def __init__(self, device=None, in_channels=None):
        super(UNetRes50, self).__init__()

        if (device is None):
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device
        
        if (in_channels is None):
            self.in_channels = 3
        else:
            self.in_channels = in_channels


        resnet50 = torchvision.models.resnet50(weights="DEFAULT")
        self.conv0 = nn.Conv2d(self.in_channels, 64, (3, 3), stride=(2, 2), padding=1)
        self.e1 = nn.Sequential(self.conv0, resnet50._modules['bn1'], resnet50._modules['relu'])
        self.e2 = nn.Sequential(resnet50._modules['maxpool'], resnet50._modules['layer1'])
        self.e3 = nn.Sequential(resnet50._modules['layer2'])
        self.e4 = nn.Sequential(resnet50._modules['layer3'])
            
        for param in self.e2.parameters():
            param.requires_grad_(False)
            
        for param in self.e3.parameters():
            param.requires_grad_(False)
            
        for param in self.e4.parameters():
            param.requires_grad_(False)
        
        self.c1 = self.convblock(1024, 512)
        self.c2 = self.convblock(512, 256)
        self.c3 = self.convblock(192, 128)
        self.c4 = self.convblock(64+self.in_channels, 64)
        
        self.u1 = nn.ConvTranspose2d(1024, 512, (2, 2), stride=(2, 2))
        self.u2 = nn.ConvTranspose2d(512, 256, (2, 2), stride=(2, 2))
        self.u3 = nn.ConvTranspose2d(256, 128, (2, 2), stride=(2, 2))
        self.u4 = nn.ConvTranspose2d(128, 64, (2, 2), stride=(2, 2))
        
        self.out = nn.Conv2d(64, 1, (1, 1), padding="same")

        self = self.to(self.device)

    def convblock(self, in_features, out_features):
        block = nn.Sequential(
            nn.Conv2d(in_features, out_features, (3, 3), padding="same"),
            nn.BatchNorm2d(out_features),
            nn.ReLU(),
            nn.Conv2d(out_features, out_features, (3, 3), padding="same"),
            nn.BatchNorm2d(out_features),
            nn.ReLU()
            )
        block = block.to(self.device)
        return block


    def forward(self, X):
        X = X.to(self.device)

        s1 = self.e1(X)
        s2 = self.e2(s1)
        s3 = self.e3(s2)
        s4 = self.e4(s3)
        
        s4 = self.u1(s4)
        s3 = self.c1(torch.cat([s3, s4], dim=1))
        s4 = None

        s3 = self.u2(s3)
        s2 = self.c2(torch.cat([s2, s3], dim=1))
        s3 = None

        s2 = self.u3(s2)
        s1 = self.c3(torch.cat([s1, s2], dim=1))
        s2 = None

        s1 = self.u4(s1)
        X = self.c4(torch.cat([X, s1], dim=1))
        s1 = None

        X = torch.sigmoid(self.out(X))

        return X


In [11]:
class CPCUNet(nn.Module):
    def __init__(self, learning_rate=None,  loss_fn=None, optimizer=None, device=None):
        super(CPCUNet, self).__init__()

        if (device is None):
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device
        
        if (learning_rate is None):
            self.learning_rate = 1e-4
        else:
            self.learning_rate = learning_rate
        
        self.net_0 = UNetRes50(in_channels=3)
        self.net_1 = UNetRes50(in_channels=5)
        self.net_2 = UNetRes50(in_channels=4)

        if (loss_fn is None):
            self.loss_fn = nn.BCELoss()
        else:
            self.loss_fn = loss_fn

        if (optimizer is None):
            self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=1e-3)
        else:
            self.optimizer = optimizer(self.parameters(), lr=self.learning_rate, weight_decay=1e-3)

        
        self = self.to(self.device)

    def forward(self, X):
        y_0 = self.net_0(X)
        
        X_polar = torch.zeros((X.shape[0], 5, X.shape[2], X.shape[3])).to(self.device)
        centers = []
        for i in range(X.shape[0]):
            centers.append(centroid(y_0[i,:,:,:]))
            interm = torch2np(y_0[i,:,:,:])
            
            X_polar[i,0:3,:,:] = np2torch(to_polar(interm, centers[i])).to(self.device)
        
        spat = torch.ones((X.shape[0], 1, X.shape[2], X.shape[3])).to(self.device)
        i_spat = (torch.cumsum(spat, dim=2) - 1) / X.shape[2]
        j_spat = (torch.cumsum(spat, dim=3) - 1) / X.shape[3]
        
        X_polar[:,3:4,:,:] = i_spat
        X_polar[:,4:,:,:] = j_spat
        
        y_1 = self.net_1(X_polar)
        
        X_final = torch.zeros((X.shape[0], 4, X.shape[2], X.shape[3]))
        X_final[:,:3,:,:] = X
        
        for i in range(X.shape[0]):
            interm = to_cart(torch2np(y_1[i,:,:,:]), centers[i])
            X_final[i,3:,:,:] = np2torch(interm).to(self.device)
        
        y_2 = self.net_2(X_final)
        
        return y_0, y_1, y_2
            
    def fit0(self, X, y):
        X, y = X.to(self.device), y.to(self.device)
        torch.cuda.empty_cache()
        self.optimizer.zero_grad()
        
        pred = self.net_0(X)

        loss = self.loss_fn(pred, y)
        loss.backward()
        self.optimizer.step()
        loss = loss.item()
        y = (y > 0.5)
        pred = (pred > 0.5)
        numt = torch.numel(y[0])
        TP = torch.sum(torch.bitwise_and(y, pred)).item() / numt
        TN = torch.sum(torch.bitwise_not(torch.bitwise_or(y, pred))).item() / numt
        FN = torch.sum(torch.bitwise_and(y, torch.bitwise_not(pred))).item() / numt
        FP = torch.sum(torch.bitwise_and(torch.bitwise_not(y), pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))
      
    def fit1(self, X, y):
        X, y = X.to(self.device), y.to(self.device)
        torch.cuda.empty_cache()
        self.optimizer.zero_grad()
        
        y_0 = self.net_0(X)
        
        X_polar = torch.zeros((X.shape[0], 5, X.shape[2], X.shape[3])).to(self.device)
        centers = []
        for i in range(X.shape[0]):
            centers.append(centroid(y_0[i,:,:,:]))
            interm = torch2np(y_0[i,:,:,:])
            
            X_polar[i,0:3,:,:] = np2torch(to_polar(interm, centers[i])).to(self.device)
        
        spat = torch.ones((X.shape[0], 1, X.shape[2], X.shape[3])).to(self.device)
        i_spat = (torch.cumsum(spat, dim=2) - 1) / X.shape[2]
        j_spat = (torch.cumsum(spat, dim=3) - 1) / X.shape[3]
        
        X_polar[:,3:4,:,:] = i_spat
        X_polar[:,4:,:,:] = j_spat
        
        pred = self.net_1(X_polar)


        loss = self.loss_fn(pred, y)
        loss.backward()
        self.optimizer.step()
        loss = loss.item()
        y = (y > 0.5)
        pred = (pred > 0.5)
        numt = torch.numel(y[0])
        TP = torch.sum(torch.bitwise_and(y, pred)).item() / numt
        TN = torch.sum(torch.bitwise_not(torch.bitwise_or(y, pred))).item() / numt
        FN = torch.sum(torch.bitwise_and(y, torch.bitwise_not(pred))).item() / numt
        FP = torch.sum(torch.bitwise_and(torch.bitwise_not(y), pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))
    
    def fit2(self, X, y):
        X, y = X.to(self.device), y.to(self.device)
        torch.cuda.empty_cache()
        self.optimizer.zero_grad()
        pred = self(X)[2]
        loss = self.loss_fn(pred, y)
        loss.backward()
        self.optimizer.step()
        loss = loss.item()
        y = (y > 0.5)
        pred = (pred > 0.5)
        numt = torch.numel(y[0])
        TP = torch.sum(torch.bitwise_and(y, pred)).item() / numt
        TN = torch.sum(torch.bitwise_not(torch.bitwise_or(y, pred))).item() / numt
        FN = torch.sum(torch.bitwise_and(y, torch.bitwise_not(pred))).item() / numt
        FP = torch.sum(torch.bitwise_and(torch.bitwise_not(y), pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))

    def test0(self, X, y):
        X, y = X.to(self.device), y.to(self.device)

        pred = self.net_0(X)

        loss = self.loss_fn(pred, y).item()
        y = (y > 0.5)
        pred = (pred > 0.5)
        numt = torch.numel(y[0])
        TP = torch.sum(torch.bitwise_and(y, pred)).item() / numt
        TN = torch.sum(torch.bitwise_not(torch.bitwise_or(y, pred))).item() / numt
        FN = torch.sum(torch.bitwise_and(y, torch.bitwise_not(pred))).item() / numt
        FP = torch.sum(torch.bitwise_and(torch.bitwise_not(y), pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))
    
    def test1(self, X, y):
        X, y = X.to(self.device), y.to(self.device)
        
        y_0 = self.net_0(X)
        
        X_polar = torch.zeros((X.shape[0], 5, X.shape[2], X.shape[3])).to(self.device)
        centers = []
        for i in range(X.shape[0]):
            centers.append(centroid(y_0[i,:,:,:]))
            interm = torch2np(y_0[i,:,:,:])
            
            X_polar[i,0:3,:,:] = np2torch(to_polar(interm, centers[i])).to(self.device)
        
        spat = torch.ones((X.shape[0], 1, X.shape[2], X.shape[3])).to(self.device)
        i_spat = (torch.cumsum(spat, dim=2) - 1) / X.shape[2]
        j_spat = (torch.cumsum(spat, dim=3) - 1) / X.shape[3]
        
        X_polar[:,3:4,:,:] = i_spat
        X_polar[:,4:,:,:] = j_spat
        
        pred = self.net_1(X_polar)

        loss = self.loss_fn(pred, y).item()
        y = (y > 0.5)
        pred = (pred > 0.5)
        numt = torch.numel(y[0])
        TP = torch.sum(torch.bitwise_and(y, pred)).item() / numt
        TN = torch.sum(torch.bitwise_not(torch.bitwise_or(y, pred))).item() / numt
        FN = torch.sum(torch.bitwise_and(y, torch.bitwise_not(pred))).item() / numt
        FP = torch.sum(torch.bitwise_and(torch.bitwise_not(y), pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))
    
    def test2(self, X, y):
        X, y = X.to(self.device), y.to(self.device)
        pred = self(X)[2]
        loss = self.loss_fn(pred, y).item()
        y = (y > 0.5)
        pred = (pred > 0.5)
        numt = torch.numel(y[0])
        TP = torch.sum(torch.bitwise_and(y, pred)).item() / numt
        TN = torch.sum(torch.bitwise_not(torch.bitwise_or(y, pred))).item() / numt
        FN = torch.sum(torch.bitwise_and(y, torch.bitwise_not(pred))).item() / numt
        FP = torch.sum(torch.bitwise_and(torch.bitwise_not(y), pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))
        
        


In [12]:
net = CPCUNet()

In [22]:
torchinfo.summary(net, input_size=(1, 3, IM_H, IM_W))

Layer (type:depth-idx)                             Output Shape              Param #
CPCUNet                                            [1, 1, 384, 512]          --
├─UNetRes50: 1-1                                   [1, 1, 384, 512]          --
│    └─Sequential: 2-1                             [1, 64, 192, 256]         --
│    │    └─Conv2d: 3-1                            [1, 64, 192, 256]         1,792
│    │    └─BatchNorm2d: 3-2                       [1, 64, 192, 256]         128
│    │    └─ReLU: 3-3                              [1, 64, 192, 256]         --
│    └─Sequential: 2-2                             [1, 256, 96, 128]         --
│    │    └─MaxPool2d: 3-4                         [1, 64, 96, 128]          --
│    │    └─Sequential: 3-5                        [1, 256, 96, 128]         (215,808)
│    └─Sequential: 2-3                             [1, 512, 48, 64]          --
│    │    └─Sequential: 3-6                        [1, 512, 48, 64]          (1,219,584)
│    └─Sequenti

In [34]:
for t in range(EPOCHS):
  torch.cuda.empty_cache()
  print(f"Epoch {t+1} ---------------------")
  print(f"Training Set -----")
  train_loss, _ = train(net, 0)
  print(f"Validation Set -----")
  train_loss, met = test(net, 0)
  print(f"\rIoU: {met[0] / (met[0] + met[1] + met[2]): .4f}, Dice: {met[0] / (met[0] + 0.5 * (met[1] + met[2])): .4f}, Acc: {met[0] + met[3] : .4f}")

Epoch 1 ---------------------
Training Set -----


Avg. Loss:  0.3673, Avg. Accuracy:  0.8848: 100%|██████████| 518/518 [07:02<00:00,  1.22it/s]


Validation Set -----


Avg. Loss:  0.3561, Avg. Accuracy:  0.8652: 100%|██████████| 20/20 [00:06<00:00,  3.22it/s]


IoU:  0.6105, Dice:  0.7582, Acc:  0.8652
Epoch 2 ---------------------
Training Set -----


...

Validation Set -----


Avg. Loss:  0.3130, Avg. Accuracy:  0.8668: 100%|██████████| 20/20 [00:06<00:00,  3.24it/s]


IoU:  0.6143, Dice:  0.7611, Acc:  0.8668
Epoch 5 ---------------------
Training Set -----


Avg. Loss:  0.1327, Avg. Accuracy:  0.9651:   0%|          | 2/518 [00:02<10:05,  1.17s/it]


KeyboardInterrupt: ignored

In [12]:
for t in range(EPOCHS):
  torch.cuda.empty_cache()
  print(f"Epoch {t+1} ---------------------")
  print(f"Training Set -----")
  train_loss, _ = train(net, 1)
  print(f"Validation Set -----")
  train_loss, met = test(net, 1)
  print(f"\rIoU: {met[0] / (met[0] + met[1] + met[2]): .4f}, Dice: {met[0] / (met[0] + 0.5 * (met[1] + met[2])): .4f}, Acc: {met[0] + met[3] : .4f}")

Epoch 1 ---------------------
Training Set -----


Avg. Loss:  0.5416, Avg. Accuracy:  0.7801:   5%|▍         | 24/518 [00:32<11:14,  1.37s/it]


KeyboardInterrupt: ignored

In [13]:
for t in range(EPOCHS):
  torch.cuda.empty_cache()
  print(f"Epoch {t+1} ---------------------")
  print(f"Training Set -----")
  train_loss, _ = train(net, 2)
  print(f"Validation Set -----")
  train_loss, met = test(net, 2)
  print(f"\rIoU: {met[0] / (met[0] + met[1] + met[2]): .4f}, Dice: {met[0] / (met[0] + 0.5 * (met[1] + met[2])): .4f}, Acc: {met[0] + met[3] : .4f}")

Epoch 1 ---------------------
Training Set -----


Avg. Loss:  0.7349, Avg. Accuracy:  0.4113:   4%|▍         | 22/518 [00:34<12:59,  1.57s/it]


KeyboardInterrupt: ignored